<a href="https://colab.research.google.com/github/shailendrarg/NLP/blob/master/NLP_Twitter_sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Twitter Sentiment Analysis

This kernel is the solution which consist of building a system that can classify tweets as Sad or Happy.

## Solution

We will start by reading some tweets so we can understand our data better. We will then try to transform our tweets into something usable by different ML models, where we are going to choose the more efficient. We will finally fine tune our model and then test it to see its efficiency on new data.


Observations:

1.I used lemmatization instead of steaming

2.I also noticed that I was mistaken when I stopped the max_features parameter at 20000 while doing GridSearch, I should have tested a bigger one, because if it stopped at 20000 (which is the max), it may get better using a bigger one. I just added None (no limit).

### Content

- [Loading the data](#Load-the-data)
- [Visualize the tweets](#Visualize-the-tweets)
    - [Emoticons](#Emoticons)
    - [Most used words](#Most-used-words)
    - [Stop words](#Stop-words)
    - [Stemming](#Stemming)
- [Prepare the data](#Prepare-the-data)
    - [Bag of Words](#Bag-of-Words)
    - [Building the pipeline](#Building-the-pipeline)
- [Select a model](#Select-a-model)
- [Fine tune the model](#Fine-tune-the-model)
- [Testing the model](#Test)
    - [Test your tweet](#Test-your-tweet)

## Load the data

In [0]:
import numpy as np
import pandas as pd

# This is for making some large tweets to be displayed
pd.options.display.max_colwidth = 100

# I got some encoding issue, I didn't knew which one to use !
# This post suggested an encoding that worked!
# https://stackoverflow.com/questions/19699367/unicodedecodeerror-utf-8-codec-cant-decode-byte
#train_data = pd.read_csv("C:\\Users\\vidya\\Downloads\\Twitter sentiment analysis\\Data\\train.csv", encoding='ISO-8859-1')
train_data = pd.read_csv("/content/drive/My Drive/Machine Learning/Twitter Sentiment analysis/twitter-sentiment-analysis2/train.csv", encoding='ISO-8859-1')

In [2]:
train_data

,ItemID,Sentiment,SentimentText
0,1,0,is so sad for my APL friend.............
1,2,0,I missed the New Moon trailer...
2,3,1,omg its already 7:30 :O
3,4,0,.. Omgaga. Im sooo im gunna CRy. I've been at this dentist since 11.. I was suposed 2...
4,5,0,i think mi bf is cheating on me!!! T_T
...,...,...,...
99984,99996,0,@Cupcake seems like a repeating problem hope you're able to find something.
99985,99997,1,"@cupcake__ arrrr we both replied to each other over different tweets at the same time , i'll se..."
99986,99998,0,@CuPcAkE_2120 ya i thought so
99987,99999,1,@Cupcake_Dollie Yes. Yes. I'm glad you had more fun with me.


# Visualize the tweets

From the tweets above, we can already make some remarks about the data:

- We can see that there is some garbage like '&amp', '&lt' (which are basically used in HTML) that aren't gonna help us in our classification
- In twitter, people mention their friends with tags like @username, there is a lot of them in our data. I was discussing with a friend about the usefulness of tags in our classification, for him, people tend to mention more friends when they are happy, but I think that people may mention people because they made bad things. When we face this kind of uncertainty, it's better to try the different options and evaluate which will do well, this is what we are gonna do.

In [3]:
# We will now take a look at random tweets
# to gain more insights

rand_indexs = np.random.randint(1,len(train_data),50).tolist()
train_data["SentimentText"][rand_indexs]

4439                                                               please!! hope and time.. nothing more! D:
32259                                                          @againtoday i'm hear ya dude, i can't either 
34213                                                          @AJlovesmusic yes i know. MIMI: STOP PLEASE, 
74795                                                          @bryanchauvel Wooo - hoo!  That is exciting!!
85025                        @cbryant68 it will be fun. PS I'm not coming to work today. I don't feel good. 
66689                                                     @britgeekgrrl every day is a bad day in my office 
81789                                                         @carolinekristek its a mutual awesomeness lol 
68822    @BStyleINC Nope will do WED O didn't tell the mac died June 18, 2009...Let's Us pray  RIP Oct. 2...
59971                                                                      @bforcefield NOOOO! It cant be!! 
77950              

#### Note
You will not have the same results at each execution because of the randomization. For me, after some execution, I noticed this:
- There is tweets with a url (like tweet 35546): we must think about a way to handle URLs, I thought about deleting them because a domain name or the protocol used will not make someone happy or sad unless the domain name is 'food.com'.
- The use of hashtags: we should keep only the words without '#' so words like python and the hashtag '#python' can be seen as the same word, and of course they are.
- Words like 'as', 'to' and 'so' should be deleted, because they only serve as a way to link phrases and words

#### Emoticons
The internet language includes so many emoticons, people also tend to create their own, so we will first analyze the emoticons included in our dataset, try to classify them as happy and said, and make sure that our model know about them.

In [4]:
# We are gonna find what emoticons are used in our dataset
import re
tweets_text = train_data.SentimentText.str.cat()
emos = set(re.findall(r" ([xX:;][-']?.) ",tweets_text))
emos_count = []
for emo in emos:
    emos_count.append((tweets_text.count(emo), emo))
sorted(emos_count,reverse=True)

[(3281, ':/'),
 (2874, 'x '),
 (2626, ': '),
 (1339, 'x@'),
 (1214, 'xx'),
 (1162, 'xa'),
 (984, ';3'),
 (887, 'xp'),
 (842, 'xo'),
 (713, ';)'),
 (483, 'xe'),
 (431, ';I'),
 (353, ';.'),
 (254, 'xD'),
 (251, 'x.'),
 (245, '::'),
 (234, 'X '),
 (217, ';t'),
 (209, ';s'),
 (185, ':O'),
 (176, ':3'),
 (166, ';D'),
 (159, ":'"),
 (157, 'XD'),
 (146, 'x3'),
 (142, ':p'),
 (126, ":'("),
 (118, ':@'),
 (117, 'xh'),
 (117, ':S'),
 (109, 'xm'),
 (104, ';p'),
 (104, ';-)'),
 (92, ':|'),
 (91, 'x,'),
 (89, ';P'),
 (76, 'xd'),
 (75, ';o'),
 (75, ';d'),
 (71, ':o'),
 (65, 'XX'),
 (63, ':L'),
 (59, 'Xx'),
 (59, ':1'),
 (58, ':]'),
 (57, ':s'),
 (56, ':0'),
 (54, 'XO'),
 (44, ';;'),
 (43, ';('),
 (38, ':-D'),
 (37, 'xk'),
 (36, 'XT'),
 (35, 'x?'),
 (35, 'x)'),
 (34, 'x2'),
 (33, ';/'),
 (32, 'x:'),
 (32, ':\\'),
 (31, 'x-'),
 (27, 'Xo'),
 (27, 'XP'),
 (27, ':-/'),
 (26, ':-P'),
 (25, ':*'),
 (23, 'xX'),
 (22, ":')"),
 (17, 'xP'),
 (16, ':['),
 (16, ':-p'),
 (14, 'x]'),
 (14, 'XM'),
 (13, ':-O'),
 (1

We should by now know which emoticons are used (and its frequency) to build two regex, one for the happy ones and another for the sad ones. We will then use them in the preprocessing process to mark them as using happy emoticons or sad ones.

In [5]:
HAPPY_EMO = r" ([xX;:]-?[dD)]|:-?[\)]|[;:][pP]) "
SAD_EMO = r" (:'?[/|\(]) "
print("Happy emoticons:", set(re.findall(HAPPY_EMO, tweets_text)))
print("Sad emoticons:", set(re.findall(SAD_EMO, tweets_text)))

Happy emoticons: {';P', 'xd', ';D', ';)', ';-)', 'x)', ';d', ';p', ':d', ':-D', 'xD', ':D', ';-D', 'XD', ':p'}
Sad emoticons: {':/', ':|', ':(', ":'("}


#### Most used words
What we are going to do next is to define a function that will show us top words, so we may fix things before running our learning algorithm. This function takes as input a text and output words sorted according to their frequency, starting with the most used word.

In [6]:
import nltk
nltk.download("punkt")
from nltk.tokenize import word_tokenize

# Uncomment this line if you haven't downloaded punkt before
# or just run it as it is and uncomment it if you got an error.
#nltk.download('punkt')
def most_used_words(text):
    tokens = word_tokenize(text)
    frequency_dist = nltk.FreqDist(tokens)
    print("There is %d different words" % len(set(tokens)))
    return sorted(frequency_dist,key=frequency_dist.__getitem__, reverse=True)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [7]:
most_used_words(train_data.SentimentText.str.cat())[:100]

There is 134028 different words


['@',
 '!',
 '.',
 'I',
 ',',
 'to',
 'the',
 'you',
 '?',
 'a',
 'it',
 'i',
 '...',
 ';',
 'and',
 '&',
 'my',
 'for',
 'is',
 'that',
 "'s",
 "n't",
 'in',
 'of',
 'me',
 'have',
 'on',
 'quot',
 "'m",
 'so',
 ':',
 'but',
 '#',
 'do',
 'was',
 'be',
 'not',
 'your',
 'are',
 'just',
 'with',
 'like',
 '-',
 'at',
 'too',
 'get',
 'good',
 'u',
 'up',
 'know',
 'all',
 'this',
 'now',
 'no',
 'we',
 'out',
 ')',
 'love',
 'can',
 '(',
 'what',
 'one',
 'will',
 'lol',
 'go',
 'about',
 'did',
 "'ll",
 'got',
 'amp',
 'there',
 'day',
 'http',
 'see',
 "'re",
 'if',
 'time',
 'they',
 'think',
 'as',
 'when',
 'from',
 'You',
 'It',
 'going',
 'really',
 'am',
 'work',
 'well',
 'had',
 'would',
 'how',
 'he',
 'here',
 'some',
 'thanks',
 'back',
 'im',
 'haha',
 'or']

#### Stop words

What we can see is that stop words are the most used, but in fact they don't help us determine if a tweet is happy/sad, however, they are consuming memory and they are making the learning process slower, so we really need to get rid of them.

In [8]:
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [9]:
from nltk.corpus import stopwords

#nltk.download("stopwords")

mw = most_used_words(train_data.SentimentText.str.cat())
most_words = []
for w in mw:
    if len(most_words) == 1000:
        break
    if w in stopwords.words("english"):
        continue
    else:
        most_words.append(w)

There is 134028 different words


In [10]:
# What we did is to filter only non stop words.
# We will now get a look to the top 1000 words
sorted(most_words)

['!',
 '#',
 '$',
 '%',
 '&',
 "'",
 "'d",
 "'ll",
 "'m",
 "'re",
 "'s",
 "'ve",
 '(',
 ')',
 '*',
 '*hugs*',
 '*sigh*',
 '+',
 ',',
 '-',
 '--',
 '.',
 '..',
 '...',
 '/',
 '1',
 '10',
 '100',
 '12',
 '1st',
 '2',
 '20',
 '2nd',
 '3',
 '30',
 '30SECONDSTOMARS',
 '4',
 '5',
 '6',
 '7',
 '8',
 ':',
 ';',
 '=',
 '?',
 '@',
 'A',
 'AND',
 'Ah',
 'AlexAllTimeLow',
 'All',
 'Also',
 'Alyssa_Milano',
 'Am',
 'And',
 'Are',
 'As',
 'At',
 'Aw',
 'Awesome',
 'Aww',
 'Awww',
 'BSB',
 'Birthday',
 'But',
 'Ca',
 'Can',
 'Chris',
 'Come',
 'Congrats',
 'Cool',
 'D',
 'DM',
 'DO',
 'Damn',
 'Day',
 'Did',
 'Do',
 'Enjoy',
 'FF',
 'Follow',
 'FollowFriday',
 'For',
 'Friday',
 'Get',
 'Glad',
 'Go',
 'God',
 'Good',
 'Got',
 'Great',
 'Had',
 'Haha',
 'Happy',
 'Have',
 'He',
 'Hello',
 'Hey',
 'Hi',
 'Hope',
 'How',
 'I',
 'IS',
 'IT',
 'If',
 'Im',
 'In',
 'Is',
 'It',
 'Its',
 'July',
 'June',
 'Just',
 'Keep',
 'LA',
 'LMAO',
 'LOL',
 'LOVE',
 'Let',
 'Like',
 'Lol',
 'London',
 'Love',
 'ME',


#### Stemming

You should have noticed something, right? There are words that have the same meaning, but written in a different manner, sometimes in the plural and sometimes with a suffix (ing, es ...), this will make our model think that they are different words and also make our vocabulary bigger (waste of memory and time for the learning process). The solution is to reduce those words with the same root, this is called stemming.

In [0]:
# I'm defining this function to use it in the 
# Data Preparation Phase
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer

#nltk.download('wordnet')
def stem_tokenize(text):
    stemmer = SnowballStemmer("english")
    stemmer = WordNetLemmatizer()
    return [stemmer.lemmatize(token) for token in word_tokenize(text)]

def lemmatize_tokenize(text):
    lemmatizer = WordNetLemmatizer()
    return [lemmatizer.lemmatize(token) for token in word_tokenize(text)]

I will stop here, but you can visualize tweets more and more to gain insights and take decisions about how to transform your data.

# Prepare the data

In this phase, we will transform our tweets into a more usable data by our ML models.

#### Bag of Words
We are going to use the Bag of Words algorithm, which basically takes a text as input, extract words from it (this is our vocabulary) to use them in the vectorization process. When a tweet comes in, it will vectorize it by counting the number of occurrences of each word in our vocabulary.

For example, we have this two tweets: "I learned a lot today" and "hahaha I got you".

|tweet / words| I | learned | a | lot | today | hahaha | got | you |
|-----|---|---------|---|-----|-------|--------|-----|-----|
| first | 1 | 1 | 1 | 1 | 1 | 0 | 0 | 0 |
| second | 1 | 0 | 0 | 0 | 0 | 1 | 1 | 1 |

We first extract the words present in the two tweets, then for each tweet we count the occurrences of each word in our vocabulary.

This is the simplest form of the Bag of Words algorithm, however, there is other variants, we are gonna use the TF-IDF (Term Frequency - Inverse Document Frequency) variant. You can read about it in the chapter I have provided in the beginning or in the official doc of scikit-learn [here](http://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction)

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer

#### Building the pipeline

It's always a good practice to make a pipeline of transformation for your data, it will make the process of data transformation really easy and reusable.
We will implement a pipeline for transforming our tweets to something that our ML models can digest (vectors).

In [0]:
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.pipeline import Pipeline

In [0]:
# We need to do some preprocessing of the tweets.
# We will delete useless strings (like @, # ...)
# because we think that they will not help
# in determining if the person is Happy/Sad

class TextPreProc(BaseEstimator,TransformerMixin):
    def __init__(self, use_mention=False):
        self.use_mention = use_mention
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        # We can choose between keeping the mentions
        # or deleting them
        if self.use_mention:
            X = X.str.replace(r"@[a-zA-Z0-9_]* ", " @tags ")
        else:
            X = X.str.replace(r"@[a-zA-Z0-9_]* ", "")
            
        # Keeping only the word after the #
        X = X.str.replace("#", "")
        X = X.str.replace(r"[-\.\n]", "")
        # Removing HTML garbage
        X = X.str.replace(r"&\w+;", "")
        # Removing links
        X = X.str.replace(r"https?://\S*", "")
        # replace repeated letters with only two occurences
        # heeeelllloooo => heelloo
        X = X.str.replace(r"(.)\1+", r"\1\1")
        # mark emoticons as happy or sad
        X = X.str.replace(HAPPY_EMO, " happyemoticons ")
        X = X.str.replace(SAD_EMO, " sademoticons ")
        X = X.str.lower()
        return X

In [15]:
nltk.download("wordnet")

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [0]:
# This is the pipeline that will transform our tweets to something eatable.
# You can see that we are using our previously defined stemmer, it will
# take care of the stemming process.
# For stop words, we let the inverse document frequency do the job
from sklearn.model_selection import train_test_split

sentiments = train_data['Sentiment']
tweets = train_data['SentimentText']

# I get those parameters from the 'Fine tune the model' part
vectorizer = TfidfVectorizer(tokenizer=lemmatize_tokenize, ngram_range=(1,2))
pipeline = Pipeline([
    ('text_pre_processing', TextPreProc(use_mention=True)),
    ('vectorizer', vectorizer),
])

# Let's split our data into learning set and testing set
# This process is done to test the efficency of our model at the end.
# You shouldn't look at the test data only after choosing the final model
learn_data, test_data, sentiments_learning, sentiments_test = train_test_split(tweets, sentiments, test_size=0.3)

# This will tranform our learning data from simple text to vector
# by going through the preprocessing tranformer.
learning_data = pipeline.fit_transform(learn_data)

# Select a model

When we have our data ready to be processed by ML models, the question we should ask is which model to use?

The answer varies depending on the problem and data, for example, it's known that Naive Bias has proven good efficacy against Text Based Problems.

A good way to choose a model is to try different candidate, evaluate them using cross validation, then chose the best one which will be later tested against our test data.

In [17]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB, MultinomialNB

lr = LogisticRegression()
bnb = BernoulliNB()
mnb = MultinomialNB()

models = {
    'logitic regression': lr,
    'bernoulliNB': bnb,
    'multinomialNB': mnb,
}

for model in models.keys():
    scores = cross_val_score(models[model], learning_data, sentiments_learning, scoring="f1", cv=10)
    print("===", model, "===")
    print("scores = ", scores)
    print("mean = ", scores.mean())
    print("variance = ", scores.var())
    models[model].fit(learning_data, sentiments_learning)
    print("score on the learning data (accuracy) = ", accuracy_score(models[model].predict(learning_data), sentiments_learning))
    print("")

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


=== logitic regression ===
scores =  [0.81083041 0.80865466 0.8080292  0.81695243 0.80608181 0.81339135
 0.80798248 0.81197721 0.80681406 0.81582439]
mean =  0.8106537995032035
variance =  1.2935213879864573e-05
score on the learning data (accuracy) =  0.8717424848554121

=== bernoulliNB ===
scores =  [0.7887456  0.7836353  0.78993178 0.79077391 0.79139383 0.79830548
 0.78656453 0.78591549 0.78540622 0.79178404]
mean =  0.7892456189864507
variance =  1.6069064781703227e-05
score on the learning data (accuracy) =  0.9029031889358784

=== multinomialNB ===
scores =  [0.80743958 0.81034099 0.80917576 0.81131025 0.80573461 0.81117318
 0.81037969 0.80414669 0.80253926 0.81255585]
mean =  0.8084795863846524
variance =  1.0212191141825659e-05
score on the learning data (accuracy) =  0.8998171219567951



None of those models is likely to be overfitting, I will choose the multinomialNB.

# Fine tune the model

I'm going to use the GridSearchCV to choose the best parameters to use. 

What the GridSearchCV does is trying different set of parameters, and for each one, it runs a cross validation and estimate the score. At the end we can see what are the best parameter and use them to build a better classifier.

In [18]:
from sklearn.model_selection import GridSearchCV

grid_search_pipeline = Pipeline([
    ('text_pre_processing', TextPreProc()),
    ('vectorizer', TfidfVectorizer()),
    ('model', MultinomialNB()),
])

params = [
    {
        'text_pre_processing__use_mention': [True, False],
        'vectorizer__max_features': [1000, 2000, 5000, 10000, 20000],
        'vectorizer__ngram_range': [(1,1), (1,2)],
    },
]
grid_search = GridSearchCV(grid_search_pipeline, params, cv=5, scoring='f1')
grid_search.fit(learn_data, sentiments_learning)
print(grid_search.best_params_)

{'text_pre_processing__use_mention': True, 'vectorizer__max_features': 20000, 'vectorizer__ngram_range': (1, 2)}


# Test

Testing our model against data other than the data used for training our model will show how well the model is generalising on new data.

### Note
We shouldn't test to choose the model, this will only let us confirm that the choosen model is doing well.

In [19]:
mnb.fit(learning_data, sentiments_learning)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [20]:
testing_data = pipeline.transform(test_data)
mnb.score(testing_data, sentiments_test)

0.7588092142547588

Not bad for my first attempt to solve a sentiment analysis problem. I will try to make it better if I got more free time.

In [21]:
# Predecting on the test.csv
sub_data = pd.read_csv("/content/drive/My Drive/Machine Learning/Twitter Sentiment analysis/twitter-sentiment-analysis2/test.csv", encoding='ISO-8859-1')
sub_learning = pipeline.transform(sub_data.SentimentText)
sub = pd.DataFrame(sub_data.ItemID, columns=("ItemID", "Sentiment"))
sub["Sentiment"] = mnb.predict(sub_learning)
print(sub)

        ItemID  Sentiment
0            1          0
1            2          0
2            3          1
3            4          0
4            5          0
...        ...        ...
299984  299996          1
299985  299997          1
299986  299998          1
299987  299999          1
299988  300000          1

[299989 rows x 2 columns]


### Test your tweet

The most exciting part ! Don't be too hard with my classifier...

In [23]:
# Just run it
model = MultinomialNB()
model.fit(learning_data, sentiments_learning)
tweet = pd.Series([input(),])
tweet = pipeline.transform(tweet)
proba = model.predict_proba(tweet)[0]
print("The probability that this tweet is sad is:", proba[0])
print("The probability that this tweet is happy is:", proba[1])

i feel so lonely
The probability that this tweet is sad is: 0.9156246686751592
The probability that this tweet is happy is: 0.08437533132484043
